In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import pickle
import json
with open('intents.json') as file:
    data = json.load(file)
try:
    x
    with open('data.pickel',"rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = [] #patterns 
    docs_y = [] #intents 

    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    #Training
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)
    
        with open("data.pickle","wb") as f:
            pickle.dump((words,labels,training,output),f)
            
# pre-processing the data

training = np.array(training)
output_row = np.array(output)
       
tf.reset_default_graph()
net = tflearn.input_data(shape=[None,len(training[0])]) 
net = tflearn.fully_connected(net, 8) #8 neurons
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax") #softmax gives probaility of each neuron that will be ouptut
net = tflearn.regression(net)

model = tflearn.DNN(net) 

# epoch =  number of times it sees the data
try:

    model.load("modelchatbot.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("modelchatbot.tflearn")
    
def bag_of_words(s,word):
    bag = [0  for _ in range(len(words))]
    s_word = nltk.word_tokenize(s)
    s_word =[stemmer.stem(word.lower()) for word in s_word]
    
    for se in s_word:
        for i, w in enumerate(word):
            if w == se:
                bag[i] = 1
    return np.array(bag)

def chat():
    print("start using chatbot press quit to exit")
    while True:
        inp = input("you:")
        if inp.lower() == "quit":
            break
        
        results = model.predict([bag_of_words(inp,words)])[0]
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        if results[results_index] > 0.7:
            for tg in data['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']
                
            print(random.choice(responses)) #choose randomly any response
        else:
            print("I didnt quite understand ,try asking another question")

chat()
            
            
                
                


INFO:tensorflow:Restoring parameters from /Users/laxmanjeergal/Desktop/ml/Chatbot/modelchatbot.tflearn
start using chatbot press quit to exit


In [2]:
!pwd

/Users/laxmanjeergal/Desktop/ml


In [12]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/laxmanjeergal/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
import pickle